
# Assignment 4 of the course “Introduction to Machine Learning” at the University of Leoben.

Author: Fotios Lygerakis

Editor: Björn Ellensohn

Semester: SS 2022/2023

## Introduction

This notebook is about solving an assignment for linear regression models. The task has to be solved without using sklearn and the following models where implemented from scratch:
- Linear Regression
    * Least Squares Regression
- Regularized Linear Regression
    * Ridge Regression
    * Lasso Regression (Bonus Task)

A template notebook was given outlining the structure needed to solve the exercise. But for my conveniance, I implemented my own procecudere on solving the assignment. I rather liked the idea of having the classes already doing the data processing on class initialization. That way it was easier for me to understand the different models and compare them.


##  Parent Class
So first I extended the parent class Predictor with the appropriate methods.
- Reading the csv
- Loading and splitting the data into training and test sets
- Normalizing the data

was the same for all the models, so the code for that moved into the parent class. Also I decided to define extra attributes for the pre-processed dataset and the raw dataset.

In the preprocess() method, I used the "1.5 x IQR rule" to find the outliers in the dataset. This can be nicely displayed by a boxplot. Those values then got handled by replacing them with the median value of the corresponding column. Also, missing values and zeros should be replaced, but in our case, none of them where found, as far as I know. 

The train_test_split() method, on the other hand, helps me to divide the dataset into the features and the target. Furthermore, the whole dataset gets split into training data and testing data by a factor of 0.8 to 0.2. This is used later for evaluating the models' performance.


### Start Coding
Import the libraries

In [2]:
import pandas as pd
import numpy as np

Create the Regression Models

In [3]:
class Predictor:
    def __init__(self, dataset):
        self.coefficients = None
        self.df = pd.read_csv(dataset)
        self.df_pp = self.preprocess(self.df)
        self.X_train_raw, self.X_test_raw, self.y_train_raw, self.y_test_raw = self.train_test_split(self.df)
        self.X_train_pp, self.X_test_pp, self.y_train_pp, self.y_test_pp = self.train_test_split(self.df_pp)

    def preprocess(self, df):
        # It is better to do this after splitting the dataset. --> need to change that
        # Handle missing values
        df.replace(0, np.nan, inplace=True)

        # Remove outliers using iqr rule:
        df_cleaned = df.copy()
        for column in df:
            q1 = df[column].quantile(q=0.25)
            q3 = df[column].quantile(q=0.75)
            med = df[column].median()

            iqr = q3 - q1
            upper_bound = q3+(1.5*iqr)
            lower_bound = q1-(1.5*iqr)

            df_cleaned[column][(df[column] <= lower_bound) | (df[column] >= upper_bound)] = df_cleaned[column].median()

        # Normalize data:
        df_normalized = df_cleaned.copy()
        for column in df_cleaned:

            mean = df_cleaned[column].mean()
            std = df_cleaned[column].std()

            df_normalized[column] = (df_cleaned[column] - mean) / std
        df_processed = df_normalized
        return df_processed

    def train_test_split(self, df, test_size=0.2):
        # Shuffle the rows of the dataset randomly
        df_randomized = df.sample(frac=1, random_state=42).reset_index(drop=True)

        # Extract the features and target variable
        X = df_randomized.drop('target', axis=1)
        y = df_randomized['target']

        # Split the dataset into training and testing sets
        split_ratio = 1 - test_size
        split_index = int(split_ratio * len(df_randomized))

        X_train = X[:split_index]
        X_test = X[split_index:]
        y_train = y[:split_index]
        y_test = y[split_index:]

        return X_train, X_test, y_train, y_test
        
    def fit(self, X, y):
        pass

    def predict(self, X):
        pass


## Child Classes
Code for the corresponding model.

### Linear Regression - Least Squares Regression

In [4]:
class LinearRegression(Predictor):
    def __init__(self, dataset):
        super().__init__(dataset)
        
    def fit(self, X, y):
        X = np.insert(X, 0, 1, axis=1)
        y = y.values.reshape(-1, 1)
        self.coefficients = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)

    def predict(self, X):
        X = np.insert(X, 0, 1, axis=1)
        return X.dot(self.coefficients)
    
    def regression_raw(self):
        # copy the values
        X_train = self.X_train_raw
        y_train = self.y_train_raw
        X_test = self.X_test_raw
        y_test = self.y_test_raw

        # implement linear regression

        # Implement the formula for the least-squares regression line
        X_train_T = np.transpose(X_train)
        beta = np.linalg.inv(X_train_T.dot(X_train)).dot(X_train_T).dot(y_train) # these are the weights

        # Train the model on the training set using the least-squares regression line
        y_pred_train = X_train.dot(beta) # the prediction on the train set

        # Evaluate the performance of the model on the testing set using metrics such as mean squared error and R-squared
        y_pred_test = X_test.dot(beta) # the prediction on the test set

        # calculate the mean squared error
        mse = np.mean((y_test - y_pred_test)**2)
        r_squared = 1 - (np.sum((y_test - y_pred_test)**2) / np.sum((y_test - np.mean(y_test))**2))

        print('Mean squared error:', mse)
        print('R-squared:', r_squared)

        # that should do

    def regression_pp(self):
        # copy the values
        X_train = self.X_train_pp
        y_train = self.y_train_pp
        X_test = self.X_test_pp
        y_test = self.y_test_pp
        
        # implement linear regression

        # Implement the formula for the least-squares regression line
        X_train_T = np.transpose(X_train)
        beta = np.linalg.inv(X_train_T.dot(X_train)).dot(X_train_T).dot(y_train) # these are the weights

        # Train the model on the training set using the least-squares regression line
        y_pred_train = X_train.dot(beta) # the prediction on the train set

        # Evaluate the performance of the model on the testing set using metrics such as mean squared error and R-squared
        y_pred_test = X_test.dot(beta) # the prediction on the test set

        # calculate the mean squared error
        mse = np.mean((y_test - y_pred_test)**2)
        r_squared = 1 - (np.sum((y_test - y_pred_test)**2) / np.sum((y_test - np.mean(y_test))**2))

        print('Mean squared error:', mse)
        print('R-squared:', r_squared)

### Regularized Linear Regression - Ridge Regression

In [5]:
class RidgeRegression(Predictor):
    def __init__(self, dataset, alpha=1):
        super().__init__(dataset) # this command initializes the parent class (Predictor) and passes the dataset.
        self.alpha = alpha
    
    def regression_raw(self):
        """
        Fits a ridge regression model on the training data using the specified regularization parameter alpha.
        Using raw dataset
        """
        X_train = self.X_train_raw
        X_test = self.X_test_raw
        y_train = self.y_train_raw
        y_test = self.y_test_raw
        alpha = self.alpha
        
        # Add a cloumn of 1s to the training data to have the correct dimension.
        X_train = np.hstack([np.ones((X_train.shape[0], 1)), X_train])

        n_features = X_train.shape[1]
        I = np.eye(n_features)
        w = np.linalg.inv(X_train.T.dot(X_train) + alpha * I).dot(X_train.T).dot(y_train)

        #X_test = (X_test - X.mean()) / X.std() #this should not be needed, since the normalization is happening in the constructor
        X_test = np.hstack([np.ones((X_test.shape[0], 1)), X_test])
        y_pred = X_test.dot(w)

        # calculate the mean squared error
        mse = np.mean((y_test - y_pred)**2)
        r_squared = 1 - (np.sum((y_test - y_pred)**2) / np.sum((y_test - np.mean(y_test))**2))

        # calculate root mean squared error (RMSE)
        rmse = np.sqrt(mse)

        print("Predicted target value:", y_pred[0])
        print("Mean squared error (MSE):", mse)
        print("R-squared:", r_squared)
        print("Root mean squared error (RMSE):", rmse)

    def regression_pp(self):
        """
        Fits a ridge regression model on the training data using the specified regularization parameter alpha.
        Using processed dataset.
        """
        X_train = self.X_train_pp
        X_test = self.X_test_pp
        y_train = self.y_train_pp
        y_test = self.y_test_pp
        alpha = self.alpha
        
        # Add a cloumn of 1s to the training data to have the correct dimension.
        X_train = np.hstack([np.ones((X_train.shape[0], 1)), X_train])

        n_features = X_train.shape[1]
        I = np.eye(n_features)
        w = np.linalg.inv(X_train.T.dot(X_train) + alpha * I).dot(X_train.T).dot(y_train)

        X_test = np.hstack([np.ones((X_test.shape[0], 1)), X_test])
        y_pred = X_test.dot(w)

        # calculate the mean squared error
        mse = np.mean((y_test - y_pred)**2)
        r_squared = 1 - (np.sum((y_test - y_pred)**2) / np.sum((y_test - np.mean(y_test))**2))

        # calculate root mean squared error (RMSE)
        rmse = np.sqrt(mse)

        print("Predicted target value:", y_pred[0])
        print("Mean squared error (MSE):", mse)
        print("R-squared:", r_squared)
        print("Root mean squared error (RMSE):", rmse)


### Regularized Linear Regression - Lasso Regression (Bonus)

In [6]:
class LassoRegression(Predictor):
    def __init__(self, dataset, alpha=1, max_iter=1000, tol=0.0001):
        super().__init__(dataset)
        self.alpha = alpha
        self.max_iter = max_iter
        self.tol = tol

    def regression_raw(self):
        """
        Fits a lasso regression model on the training data using the specified regularization parameter alpha, iterations, and tolerance.
        Using raw dataset.
        """
        # Define hyperparameters
        alpha = self.alpha  # regularization strength
        max_iterations = self.max_iter  # number of gradient descent iterations
        tolerance = self.tol

        # Load the data
        diabetes = pd.read_csv("diabetes.csv")

        diabetes.insert(0, "Intercept", 1)

        train_size = int(0.8 * len(diabetes))
        
        X_train = diabetes.iloc[:train_size, :-1].values
        y_train = diabetes.iloc[:train_size, -1].values
        X_test = diabetes.iloc[train_size:, :-1].values
        y_test = diabetes.iloc[train_size:, -1].values

        theta_lasso = np.zeros(X_train.shape[1])
        for i in range(max_iterations):
            theta_prev = theta_lasso.copy()
            for j in range(X_train.shape[1]):
                if j == 0:
                    theta_lasso[j] = np.mean(y_train)
                else:
                    xj = X_train[:, j]
                    rj = y_train - X_train @ theta_lasso + xj * theta_lasso[j]
                    zj = xj @ xj
                    if zj == 0:
                        theta_lasso[j] = 0
                    else:
                        if np.sum(xj * rj) > alpha / 2:
                            theta_lasso[j] = (np.sum(xj * rj) - alpha / 2) / zj
                        elif np.sum(xj * rj) < - alpha / 2:
                            theta_lasso[j] = (np.sum(xj * rj) + alpha / 2) / zj
                        else:
                            theta_lasso[j] = 0
            if np.sum((theta_lasso - theta_prev) ** 2) < tolerance:
                break
        
        sst = np.sum((y_test - np.mean(y_test)) ** 2)

        y_pred_lasso = X_test @ theta_lasso
        mse_lasso = np.mean((y_test - y_pred_lasso) ** 2)
        ssr_lasso = np.sum((y_pred_lasso - np.mean(y_test)) ** 2)
        r_squared_lasso = 1 - (ssr_lasso / sst)

        rmse_lasso = np.sqrt(mse_lasso)

        print("Lasso regression:")
        print("Mean squared error (MSE):", mse_lasso)
        print("R-squared:", r_squared_lasso)
        print("Root mean sqaured error (RMSE):", rmse_lasso)

    def regression_pp(self):
        """
        Fits a lasso regression model on the training data using the specified regularization parameter alpha, iterations, and tolerance.
        Using processed dataset.
        """
        # Define hyperparameters
        alpha = self.alpha  # regularization strength
        max_iterations = self.max_iter  # number of gradient descent iterations
        tolerance = self.tol

        # Load the data
        diabetes_norm = pd.read_csv("diabetes_norm.csv")

        diabetes_norm.insert(0, "Intercept", 1)

        train_size = int(0.8 * len(diabetes_norm))

        X_train = diabetes_norm.iloc[:train_size, :-1].values
        y_train = diabetes_norm.iloc[:train_size, -1].values
        X_test = diabetes_norm.iloc[train_size:, :-1].values
        y_test = diabetes_norm.iloc[train_size:, -1].values

        theta_lasso = np.zeros(X_train.shape[1])
        for i in range(max_iterations):
            theta_prev = theta_lasso.copy()
            for j in range(X_train.shape[1]):
                if j == 0:
                    theta_lasso[j] = np.mean(y_train)
                else:
                    xj = X_train[:, j]
                    rj = y_train - X_train @ theta_lasso + xj * theta_lasso[j]
                    zj = xj @ xj
                    if zj == 0:
                        theta_lasso[j] = 0
                    else:
                        if np.sum(xj * rj) > alpha / 2:
                            theta_lasso[j] = (np.sum(xj * rj) - alpha / 2) / zj
                        elif np.sum(xj * rj) < - alpha / 2:
                            theta_lasso[j] = (np.sum(xj * rj) + alpha / 2) / zj
                        else:
                            theta_lasso[j] = 0
            if np.sum((theta_lasso - theta_prev) ** 2) < tolerance:
                break
        
        sst = np.sum((y_test - np.mean(y_test)) ** 2)

        y_pred_lasso = X_test @ theta_lasso
        mse_lasso = np.mean((y_test - y_pred_lasso) ** 2)
        ssr_lasso = np.sum((y_pred_lasso - np.mean(y_test)) ** 2)
        r_squared_lasso = 1 - (ssr_lasso / sst)

        rmse_lasso = np.sqrt(mse_lasso)

        print("Lasso regression:")
        print("Mean squared error (MSE):", mse_lasso)
        print("R-squared:", r_squared_lasso)
        print("Root mean sqaured error (RMSE):", rmse_lasso)


## Testing the Models

### Least Squares Regression

In [10]:
rg = LinearRegression('diabetes.csv')
# Raw Dataset
rg.regression_raw()

Mean squared error: 3460.083533525722
R-squared: 0.37854964415469516


In [11]:
# Pre-processed Dataset
rg.regression_pp()

Mean squared error: 0.5359087499312691
R-squared: 0.42794122908248045


### Ridge Regression

In [12]:
rr = RidgeRegression('diabetes.csv')
# Raw Dataset
rr.regression_raw()

Predicted target value: 208.82139795491315
Mean squared error (MSE): 3369.602933909638
R-squared: 0.39480046592928286
Root mean squared error (RMSE): 58.04828105904289


In [13]:
# Pre-processed Dataset
rr.regression_pp()

Predicted target value: 0.7839100243805647
Mean squared error (MSE): 0.5407373522105846
R-squared: 0.42278691076707886
Root mean squared error (RMSE): 0.7353484563188969


### Lasso Regression

In [14]:
lr = LassoRegression('diabetes.csv')
# Raw Dataset
lr.regression_raw()

Lasso regression:
Mean squared error (MSE): 3574.2278339827644
R-squared: 0.5172053348571604
Root mean sqaured error (RMSE): 59.784846190174015


In [15]:
# Pre-processed Dataset
lr.regression_pp()

Lasso regression:
Mean squared error (MSE): 0.5080018069655786
R-squared: 0.5404721520708679
Root mean sqaured error (RMSE): 0.7127424548640123
